In [2]:
from cobra.io import read_sbml_model
from pathlib import Path
import os

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'models/cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'models/cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]
    
mod = models[k[0]].copy()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp7a81zf2g.lp
Reading time = 0.03 seconds
: 4348 rows, 10790 columns, 44678 nonzeros


In [3]:
mod

Name,Human1
Memory address,1e44e9061a0
Number of metabolites,4348
Number of reactions,5395
Number of genes,2065
Number of groups,142
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Lysosome, Nucleus, Inner mitochondria"


In [3]:
mod.reactions.get_by_id('MAR13082').build_reaction_string(use_metabolite_names = True)

'45.0 ATP + 0.0267 DNA + 45.0 H2O + 0.1124 RNA + 0.4062 glycogen + 0.0012 cofactor_pool_biomass + 5.3375 protein_pool_biomass + 0.2212 lipid_pool_biomass + 0.4835 metabolite_pool_biomass --> 45.0 ADP + 45.0 H+ + 45.0 Pi + biomass'

In [4]:
mod.reactions.get_by_id('MAR13082').reaction

'45.0 MAM01371c + 0.0267 MAM01721n + 45.0 MAM02040c + 0.1124 MAM02847c + 0.4062 MAM03161c + 0.0012 MAM10012c + 5.3375 MAM10013c + 0.2212 MAM10014c + 0.4835 MAM10015c --> 45.0 MAM01285c + 45.0 MAM02039c + 45.0 MAM02751c + MAM03970c'

In [5]:
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

In [6]:
def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [7]:
def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model

# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [8]:
# add RPE-PR interface reactions to RPE model (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
            r.compartment = 'e_RPE-PR'
    mod_RPE.add_reactions([rxn])

In [9]:
# add RPE-PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
            r.compartment = 'e_RPE-PR'

In [10]:
# fuse RPE and PR model
from cobra import Model
mod_RPE_PR = Model('RPE-PR model')
mod_RPE_PR.add_reactions(mod_RPE.reactions)
mod_RPE_PR.add_reactions(mod_PR.reactions)

In [11]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE-PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE-PR) 
# of the reactions that only involve the e_RPE-PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE-PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE-PR' in r.id:
        r.id = r.id.replace('_RPE-PR','_eRPE-PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR-RPE','_eRPE-PR')

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [12]:
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): 
        return reaction.annotation['vmhreaction']
get_vmh_id(mod_RPE_PR.reactions[0])

'ALCD2if'

In [13]:
df= pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions],\
             columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])
df.to_clipboard(excel=True, sep=None)
df

,id,vmh id,name,subsystem,reaction (id),reaction (name)
0,MAR03905,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c + MAM02552c --> MAM01249c + MAM02039...,ethanol + NAD+ --> acetaldehyde + H+ + NADH
1,MAR03907,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c + MAM02554c --> MAM01249c + MAM02039...,ethanol + NADP+ --> acetaldehyde + H+ + NADPH
2,MAR04097,ACS,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,MAM01252c + MAM01371c + MAM01597c --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi
3,MAR04099,ACSm,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,MAM01252m + MAM01371m + MAM01597m --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi
4,MAR04108,r0068,acetyl adenylate:CoA acetyltransferase,Glycolysis / Gluconeogenesis,MAM01257c + MAM01597c --> MAM01261c + MAM01334...,acetyl adenylate + CoA --> acetyl-CoA + AMP + H+
...,...,...,...,...,...,...
13065,MAR10132,None,,Exchange/demand reactions,MAM10041e <=>,tauro-hyodeoxycholic acid <=>
13066,MAR13083,None,,Metabolism of other amino acids,MAM02039c + MAM02157c + MAM02553c + MAM02630c ...,H+ + hypotaurine + NADH + O2 --> H2O + NAD+ + ...
13067,MAR13084,None,,Transport reactions,MAM01639c <=> MAM01639n,dAMP <=> dAMP
13068,MAR13085,None,,Transport reactions,MAM01686c <=> MAM01686n,dGMP <=> dGMP


In [14]:
import pandas as pd
df = pd.DataFrame([[r.id, get_vmh_id(r), r.name, r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
                   for r in mod_RPE_PR.reactions if r.compartments == {'e_RPE-PR'}], \
                   columns=['id','vmh id', 'name', 'reaction (id)', 'reaction (name)'])
df[~df['reaction (id)'].duplicated(keep=False)].to_clipboard(excel=True, sep=None)

In [15]:
df = pd.DataFrame([[r.id, get_vmh_id(r), r.name, r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
                   for r in mod_RPE_PR.reactions], \
                   columns=['id','vmh id', 'name', 'reaction (id)', 'reaction (name)'])
df_dup = pd.DataFrame([r.reaction for r in mod_RPE_PR.reactions]).duplicated(keep='first')

In [16]:
mod.reactions.get_by_id('MAR01862')

Reaction identifier,MAR01862
Name,
Memory address,0x25037c68d90
Stoichiometry,MAM01988e + MAM02519e --> MAM01988c + MAM02519c glycocholate + Na+ --> glycocholate + Na+
GPR,ENSG00000100652 or ENSG00000125255
Lower bound,0.0
Upper bound,1000.0


In [17]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df = pd.DataFrame(index=subsystems)
for i in range(len(files)):
    count = [[rxn.subsystem for rxn in models[files[i]].reactions].count(s) for s in subsystems]
    df[model_n2[i]] = count 
df

NameError: name 'model_n2' is not defined

In [ ]:
atp_rxns = [list(m.reactions) for m in mod.metabolites if 'ATP' == m.name] 
atp_rxns = list(set([i for atp_rxns_i in atp_rxns for i in atp_rxns_i])) # unique 
df_atp_rxns = pd.DataFrame([[r.id,\
              get_vmh_id(r), \
              r.name, \
              r.build_reaction_string(use_metabolite_names = True)]\
              for r in atp_rxns])
df_atp_rxns

In [ ]:
len(atp_rxns)

In [ ]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df_h1 = pd.DataFrame(index=subsystems)
count = [[rxn.subsystem for rxn in mod.reactions].count(s) for s in subsystems]
df_h1['Human1'] = count 
df['Human1']=df_h1
df

In [ ]:
df['Human1']=df_h1

df2=df[(df == 0).any(axis=1)] # select rows and columns containing zeros
idx_row = df2[df2==0].count(axis=1).sort_values(ascending=False).index.to_list()
idx_col = df2[df2==0].count(axis=0).sort_values(ascending=False).index.to_list()
df2 = df2.reindex(idx_col, axis=1).reindex(idx_row,axis=0)

def style_negative(v, props=''):
    return props if v == 0 else None
df2 = df2[(df == 0).any(axis=1)].style.applymap(style_negative, props='background-color:gray;').\
set_table_styles([dict(selector="th", props=[('width', '10px')]),
              dict(selector="th.col_heading",
                   props=[("writing-mode", "vertical-rl"),
                          ('transform', 'rotateZ(180deg)'), 
                          ('height', '170px'),
                          ('vertical-align', 'top')]),
              dict(selector="th.row_heading",
                   props=[('width', '450px'),])])
df2

In [ ]:
df_norm = df.drop(['Human1'], axis=1).divide(df['Human1'].values, axis='rows')
idx = df_norm.mean(axis=1).sort_values(ascending=False).index.to_list()
df_norm['Human1 (number of reactions)']=df['Human1']
df_norm=df_norm.reindex(idx,axis=0)
df_norm.to_clipboard(excel=True, sep=None)
df_norm

In [ ]:
[r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
pd.DataFrame([[r.id, get_vmh_id(r), r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'],\
             columns=['id','vhr id','name','reaction','reaction (names)']).to_clipboard(excel=True, sep=None)

In [ ]:
rxns_mod = [r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
l_bin_gly_rxns = [[1 if r in rxns_i else 0 for r in rxns_mod]\
                  for rxns_i in [[r.id for r in models[f].reactions\
                  if r.subsystem == 'Glycolysis / Gluconeogenesis'] for f in files]]
df_gly_rxns = pd.DataFrame([[r.id,r.name] for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'], columns=['id','name'])
for i in range(len(model_n2)):
    df_gly_rxns[model_n2[i]]=l_bin_gly_rxns[i]

idx_row = df_gly_rxns[df_gly_rxns==0].count(axis=1).sort_values(ascending=True).index.to_list()
idx_col = df_gly_rxns[df_gly_rxns==0].count(axis=0).sort_values(ascending=True).index.to_list()
df_gly_rxns = df_gly_rxns.reindex(idx_col, axis=1).reindex(idx_row,axis=0)
df_gly_rxns.to_clipboard(excel=True, sep=None)
df_gly_rxns

In [ ]:
rxns_mod = [r.id for r in list(set([r for r in list(mod.metabolites.get_by_id('MAM01371c').reactions)]))]
l_bin_atp_rxns = [[1 if r in rxns_i else 0 for r in rxns_mod]\
                  for rxns_i in [[r.id for r in models[f].reactions] for f in files]]
df_atp_rxns = pd.DataFrame([[r.id,r.name] for r in mod.reactions], columns=['id','name'])
                            
for i in range(len(model_n2)):
    df_atp_rxns[model_n2[i]]=l_bin_atp_rxns[i]

idx_row = df_atp_rxns[df_atp_rxns==0].count(axis=1).sort_values(ascending=True).index.to_list()
idx_col = df_atp_rxns[df_atp_rxns==0].count(axis=0).sort_values(ascending=True).index.to_list()
df_atp_rxns = df_atp_rxns.reindex(idx_col, axis=1).reindex(idx_row,axis=0)
df_atp_rxns.to_clipboard(excel=True, sep=None)
df_atp_rxns